In [6]:
import json
import itertools
import math
import csv
from tqdm import tqdm
import os

### File finder

In [7]:
def find_the_way(path,file_format,con=""):
    files_add = []
    # r=root, d=directories, f = files
    for r, d, f in os.walk(path):
        for file in f:
            if file_format in file:
                if con in file:
                    files_add.append(os.path.join(r, file))  
            
    return files_add




### distance calculation function

In [8]:
def is_global_coordinate(coord):
    """
   Checks whether the coordinates are global.
   - If the latitude is between (-90, 90) and the longitude is between (-180, 180), they are global.
   - If both coordinates are above 90, they are considered local X/Y.
    """
    lat, lon = coord
    if -90 <= lat <= 90 and -180 <= lon <= 180:
        return True  # Küresel koordinat (lat/lon)
    if lat > 90 and lon > 90:
        return False  # Yerel koordinat (X/Y)
    
    # Şüpheli durumlarda uyarı ver
    print(f"UYARI: Koordinatlar tam tespit edilemedi, varsayılan olarak yerel kabul ediliyor! {coord}")
    return False  # Varsayılan olarak yerel kabul et


def haversine_distance(lat1, lon1, lat2, lon2):
    """Haversine formülü ile iki küresel koordinat arasındaki mesafeyi hesaplar (km cinsinden)."""
    R = 6371  # Dünya'nın yarıçapı (km)
    
    # Dereceden radyana çevirme
    lat1, lon1, lat2, lon2 = map(math.radians, [lat1, lon1, lat2, lon2])
    
    # Farklar
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    
    # Haversine formülü
    a = math.sin(dlat / 2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon / 2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    
    return R * c  # Sonuç km cinsinden döndürülür

def euclidean_distance(x1, y1, x2, y2):
    """Öklidyen mesafe hesaplar (yerel koordinatlar için)."""
    return math.sqrt((x2 - x1) ** 2 + (y2 - y1) ** 2)

def calculate_distance(coord1, coord2):
    """
    Koordinatların türüne göre uygun mesafe hesaplama yöntemi seçer.
    - Eğer her iki koordinat da lat/lon ise Haversine
    - Eğer her ikisi de x/y ise Öklidyen
    - Karışık girişlerde None döndürerek işlemi atlar.
    """
    is_global_1 = is_global_coordinate(coord1)
    is_global_2 = is_global_coordinate(coord2)

    if is_global_1 and is_global_2:
        return haversine_distance(*coord1, *coord2), "Haversine"
    elif not is_global_1 and not is_global_2:
        return euclidean_distance(*coord1, *coord2), "Euclidean"
    else:
        return None, "Geçersiz"  # None döndürerek hatayı önle


def intersection_count(list1, list2):
    """İki listenin ortak eleman sayısını döndürür."""
    return len(set(list1) & set(list2))


In [9]:
prelist=find_the_way("./",".json","")
files_add =  dict(zip(prelist[1::2], prelist[0::2]))
for i in files_add:
    print(i)

./data\1\Test_rss.json
./data\1\Training_rss.json
./data\2\SAH1_test_rss.json
./data\2\TIE1_test_rss.json


### Calculate the distance between fingerprints (based on coordinates)

In [10]:
for file in files_add:
    # JSON dosyalarını oku
    with open(file, "r") as f:
         rss= json.load(f)


    with open(files_add[file], "r") as f:
        data  = json.load(f)

        
    # Verileri uygun formata çevir (Anahtar: Nokta ID, Değer: (x, y) veya (lat, lon))
    points = {int(k): tuple(v) for k, v in data.items()}
    # CSV dosyasını aç ve verileri anlık olarak yaz
    with open(file.replace(".json","@.csv"), "w", newline="") as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(["point1", "point2", "distance", "method"])  # Başlık satırı


        total = len(points) * (len(points) - 1) // 2  # kombinasyon sayısı

        for (i, point1), (j, point2) in tqdm(itertools.combinations(points.items(), 2), total=total):

                list1 = list(rss[str(i)].keys())
                list2 = list(rss[str(j)].keys())
        
                if intersection_count(list1, list2):  # Ortak veri varsa mesafeyi hesapla
                    try:
                        distance = euclidean_distance(point1[0],point1[1],point2[0],point2[1])
                        writer.writerow([i, j, round(distance, 2), "euclidean_distance"])  # Dosyaya yaz
                    except ValueError as e:
                        print(f"Hata: {e}")
    print(f"{file}  kaydedildi.")


100%|███████████████████████████████████████████████████████████████████████| 481671/481671 [00:05<00:00, 84490.02it/s]


./data\1\Test_rss.json  kaydedildi.


100%|█████████████████████████████████████████████████████████████████████████| 99235/99235 [00:01<00:00, 83543.07it/s]


./data\1\Training_rss.json  kaydedildi.


100%|█████████████████████████████████████████████████████████████████████████| 12090/12090 [00:00<00:00, 86874.59it/s]


./data\2\SAH1_test_rss.json  kaydedildi.


100%|███████████████████████████████████████████████████████████████████████████| 1225/1225 [00:00<00:00, 59121.61it/s]

./data\2\TIE1_test_rss.json  kaydedildi.
